# Práctica de Programación en Entornos de Datos
## Curso 2021-2022
# Autor: Jesús Galán Llano
## Correo: jgalan279@alumno.uned.es

Este notebook contiene la práctica de la asignatura Programación en Entornos de Datos. Los apartados del enunciado se han incluído en el notebook con el fin de facilitar
su comprensión y la localización de cada punto resuelto. Las celdas con código tienen asociado una celda de texto en la que se explica el funcionamiento del código
y las justificaciones de implementación tomadas.

En primer lugar, se crea un entorno virtual para la práctica llamado "ped". La versión de Python empleada se corresponde con la 3.8.10. 

In [33]:
# ! python3 -m virtualenv ped

Instalamos las librerías necesarias. En este caso solamente hemos necesitado instalar la librería pandas

In [34]:
# ! pip3 install pandas

El resto de librería empleadas forman parte del estándar de Python. Para más información se crea un archivo requirements.txt que contiene
una lista de todas las librerías instaladas en el entorno virtual y su correpondiente versión.

In [35]:
! pip3 freeze > requirements.txt

Para instalar las librerías en el archivo requirements.txt simplemente hace falta ejecutar el siguiente comando:

In [36]:
# ! pip3 install -r requirements.txt

En primer lugar, importamos las las funciones o librerías necesarias. Son las siguientes:
 - csv: crear el archivo en formato csv
 - os: acceder a los archivos de un directorio
 - pandas: librería principal para resolver la práctica
 - Counter: clase útil para transformar variables en contadores

In [37]:
import csv
import os
import random

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from typing import List

from collections import Counter

Definir una variable constante que sea un diccionario para facilitar la conversión de notas extrañas. Las notas extrañas son E#, que se tiene 
que transformar por un F y B# que se transforma por un C. 

In [38]:
NOTES_CONS = {
    'E#': 'F',
    'B#': 'C'
}

1. Construir un fichero CSV de 90 columnas en el que en cada línea tengamos la siguiente información:
La primera línea contiene los nombres de las columnas:
    ▪ Primera columna: TITULO.
    ▪ Segunda columna: AUTOR.
    ▪ Siguientes 88 columnas: los nombres de las notas (en la notación descrita) de las 88 teclas del piano, ordenadas desde la más grave (recordemos que es A0) hasta la más aguda (que sería C8).

Este punto se ha desarrollado en varios métodos pequeños para facilitar su implementación. Estos métodos son: build_csv, load_files, load_data y transfor_notes.
El método build_csv se encarga de crear el archivo partituras.csv, que almacenará toda la información de las partituras, así como su formato.
En este punto cabe destacar cómo se han creado las columnas correspondientes a las notas de una forma sencilla. Debido a que no todas las octavas tienen el mismo número de notas estas se han dividido en tres categorías en función de su posición en el piano: inicial, medio o final. Las notas iniciales se corresponden con las primeras notas del piano que no forman parte de la primera octava, que son: A, A# y B. Las notas del medio se corresponden con las notas que forman una octava y la nota del final es la nota más a la derecha del piano que es C.

Utilizando la compresión de listas y el método range se han creado todas las notas del piano. El método range permite iterar sobre un rango, por ejemplo, para crear las 7 octavas se itera desde el número 1 hasta el número 8, que no está incluido. 

Una vez que están todas las columnas creadas se escriben en el fichero con el método writerow.

In [39]:
def build_csv() -> List[str]:
    '''Función que crea el archivo csv y configura su formato

    :return: lista de columnas que forman el fichero csv
    '''
    columns = []
    with open('partituras.csv', 'w', encoding='UTF-8', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|')
        fields = ['TITULO', 'AUTOR']
        start_notes = ['A', 'A#', 'B']
        middle_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        end_notes = ['C']
        start_notes_notation = [f'{note}{i}' for i in range(0,1) for note in start_notes]
        middle_notes_notation = [f'{note}{i}' for i in range(1,8) for note in middle_notes]
        end_notes_notation = [f'{note}{i}' for i in range(8,9) for note in end_notes]
        columns = [*fields, *start_notes_notation, *middle_notes_notation, *end_notes_notation]
        writer.writerow(columns)

    return columns

El método load_data se encarga de procesar una partitura, un archivo en formato txt, en un diccionario. En primer lugar, comprueba que el fichero existe antes de procesarlo. Tras esto se leen los datos del fichero. Las dos primeras filas se corresponden con el título y el autor de la partitura, mientras que el resto de filas se corresponden con las notas. Para obtener únicamente las notas de la partitura utilizamos la técnica de slicing y establecemos que queremos obtener todas las líneas desde la tercera, incluida, en adelante. 

Para cada nota hay que hacer una transformación debido a la posible existencia de notas extrañas como #E y #B. La función map permite aplicar una función a un conjunto de datos. En este caso ejecutamos la función transfor_notes que se encarga de convertir las notas mencionadas. 

Una vez transformadas todas las notas transformamos la lista en un diccionario, donde la clave es la nota y el valor es el número de veces que aparece, con la clase Counter. Por último, devolvemos este diccionario.

In [40]:
def load_data(path: str) -> dict:
    """ Método que carga los datos de un archivo y los devuelve en un diccionario

    :param str: nombre del fichero de entrada
    :return dict: dicccionario <nota, nº de veces que aparece>

    """
    if os.path.isfile(path):
        with open(path, 'r', encoding='UTF-8') as file:
            lines = file.read().splitlines()
            # Transformar las notas utilizando la función map
            notes_list = list(map(transfor_notes, lines[3:]))
            # Utilizar funcion count para contar cuantas veces aparece cada nota en la partitura
            notes_dict = dict(Counter(notes_list))
            notes_dict['TITULO'] = lines[0]
            notes_dict['AUTOR'] = lines[1]
            
        return notes_dict

El método transfor_notes se encarga de transformar las notas de la partitura para corregir la presencia de las denominadas notas extrañas.
Recibe como parámetro la nota en forma de string y retorna la nota resultante tras aplicar la transformación. 

Las notas extrañas se corresponden con E# y B#, que son notas que no tienen su correspondiente tecla en el piano pero musicalmente tienen sentido. Para transforma una nota primero obtenemos su tecla y su octava. Como las notas siempre van a ser sostenidas van a tener siempre 2 caracteres más otro caracter que indica la octava. Si la nota se corresponde con B# sumamos uno a su octava. La transformación de la nota se realiza utilizando el diccionario NOTES_CONS. Se ha decidido utilizar un diccionario y no una serie de if, elif, elese encadenados porque la transformación es más sencilla, el código es más limpio y computacionalmente es más eficiente porque la búsqueda en el diccionario es O(1). 

In [41]:
def transfor_notes(note: str) -> str:
    """ Método que transforma las notas en caso de ser extrañas
    :param note: nota a transformar

    :return: nota transformada
    """
    if 'E#' in note or 'B#' in note:
        note, octave = note[:2], note[2]
        if note == 'B#':
            octave = int(octave) + 1
        note = NOTES_CONS[note]
        note = f'{note}{octave}'
    return note

El método load_files se encarga de leer todas las partituras del directorio pasado como parámetro y de escribir los datos en el archivo csv. En el caso de que no se incluya un directorio se tiene en encuenta el directorio actual desde donde se ejecuta el notebook.
Este método se encarga de llamar al método anterior para crear el fichero csv y, una vez creado, procesa todos los ficheros del directorio. Para cada fichero se cargan sus datos con el método load data y su resultado se transforma en un dataframe de pandas. Por último, el dataframe se escribe en el fichero csv con el método to_csv. 
Se ha decidido establecer un valor 0 para aquellas notas válidas que no aparecen en una partitura con el fin de que las filas del archivo tengan el mismo formato a pesar de sus distintas características.

In [42]:
def load_files(dir: str = '.') -> None:
    """ Método que carga los ficheros de un directorio en un archivo csv

    El directorio de origen se pasa como parámetro. En caso de que se omita se utiliza el directorio actual
    :param columns: columnas del archivo csv
    :param dir: ruta del directorio origen
    :return: None
    """
    if not os.path.isdir(dir):
        return
    columns = build_csv()
    input_files = os.listdir(dir)
    for file in input_files:
        file = os.path.join(dir, file)
        print(f'Procesando fichero {file}\n')
        result = load_data(file)
        df = pd.DataFrame([result], columns=columns).fillna(0)
        df.to_csv('partituras.csv', mode='a', header=False, index=None)

Por último, ejectuamos el programa invocando al método load_files. Este proceso se realiza dentro de un bloque try/catch para controlar las posibles excepciones que pudieran aparecer durante la ejecución. 

In [43]:
try:
    load_files(dir='PARTITURAS')

except Exception as e:
    print(e)

finally:
    print('Ejecución finalizada')

Procesando fichero PARTITURAS/opus21.txt

Procesando fichero PARTITURAS/opus22.txt

Procesando fichero PARTITURAS/opus24.txt

Procesando fichero PARTITURAS/opus13.txt

Procesando fichero PARTITURAS/opus16.txt

Procesando fichero PARTITURAS/opus06.txt

Procesando fichero PARTITURAS/opus02.txt

Procesando fichero PARTITURAS/opus10.txt

Procesando fichero PARTITURAS/opus14.txt

Procesando fichero PARTITURAS/opus11.txt

Procesando fichero PARTITURAS/opus12.txt

Procesando fichero PARTITURAS/opus05.txt

Procesando fichero PARTITURAS/opus20.txt

Procesando fichero PARTITURAS/opus04.txt

Procesando fichero PARTITURAS/opus18.txt

Procesando fichero PARTITURAS/opus25.txt

Procesando fichero PARTITURAS/opus15.txt

Procesando fichero PARTITURAS/opus19.txt

Procesando fichero PARTITURAS/opus26.txt

Procesando fichero PARTITURAS/opus23.txt

Procesando fichero PARTITURAS/opus17.txt

Procesando fichero PARTITURAS/opus08.txt

Procesando fichero PARTITURAS/opus09.txt

Procesando fichero PARTITURAS/opus

2. Escribe el código que permita cargar un CSV como el creado en el ejercicio 1 en un
DataFrame de Pandas. Trata de optimizar el espacio utilizado, incluye todas las
explicaciones que consideres necesarias. El resto de ejercicios de la práctica realizarán su
trabajo partiendo del DataFrame cargado en este ejercicio.

Comprobamos que la lectura del fichero es correcta leyendo el fichero csv generado. El método read_csv permite cargar el fichero en un dataframe de pandas.
Se utilizan varios parámetros para optimizar la carga y el uso del daframae:
 * engine: establece el mecanismo de análisis sitáctico de Python
 * encoding: establece la codificación del fichero, útil si hay caracteres no presentes en ASCII
 * low_memory: optimiza el procesamiento del fichero diviviéndolo en trozos más pequeños.
 * memory_map: optimiza el acceso a los datos porque almacena el fichero en memoria

In [44]:
df = pd.read_csv(filepath_or_buffer='partituras.csv', engine='python', encoding='utf-8', low_memory=True, memory_map=True)

In [45]:
df

,TITULO,AUTOR,A0,A#0,B0,C1,C#1,D1,D#1,E1,F1,F#1,G1,G#1,A1,A#1,B1,C2,C#2,D2,D#2,E2,F2,F#2,G2,G#2,A2,A#2,B2,C3,C#3,D3,D#3,E3,F3,F#3,G3,G#3,A3,A#3,B3,C4,C#4,D4,D#4,E4,F4,F#4,G4,G#4,A4,A#4,B4,C5,C#5,D5,D#5,E5,F5,F#5,G5,G#5,A5,A#5,B5,C6,C#6,D6,D#6,E6,F6,F#6,G6,G#6,A6,A#6,B6,C7,C#7,D7,D#7,E7,F7,F#7,G7,G#7,A7,A#7,B7,C8
0,"The Seasons, Opus 37a - April - Snowdrop",Tchaikovsky,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,10.0,2.0,5.0,4.0,21.0,25.0,2.0,36.0,2.0,49.0,17.0,14.0,77.0,15.0,115.0,20.0,110,97.0,5,57.0,50.0,128,77.0,46.0,149.0,16.0,38.0,14.0,47,23.0,4.0,17.0,9.0,23.0,8.0,4.0,8.0,0.0,0.0,1.0,14.0,9.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"The Seasons, Opus 37a - May - Starlight Nights",Tchaikovsky,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,15.0,12.0,2.0,21.0,3.0,6.0,4.0,28.0,42.0,8.0,35.0,12.0,56.0,55.0,11.0,58,22.0,58,23.0,48.0,96,5.0,35.0,19.0,85.0,72.0,4.0,56,37.0,68.0,11.0,25.0,64.0,9.0,23.0,5.0,16.0,14.0,1.0,12.0,2.0,4.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"The Seasons, Opus 37a - July - Song of the Reaper",Tchaikovsky,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,15.0,0.0,1.0,1.0,0.0,4.0,3.0,24.0,1.0,0.0,48.0,0.0,25.0,0.0,15.0,60.0,0.0,24.0,3.0,20.0,6.0,44,102.0,5,72.0,4.0,59,97.0,0.0,33.0,7.0,108.0,34.0,14,71.0,11.0,51.0,2.0,34.0,41.0,0.0,27.0,4.0,28.0,6.0,2.0,8.0,2.0,7.0,2.0,5.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Piano Sonata in C major, Hoboken XVI:7 - Movem...",Haydn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,24.0,0.0,20.0,11.0,12.0,10.0,2.0,51.0,0.0,4,0.0,22,33.0,0.0,21,6.0,12.0,7.0,4.0,45.0,0.0,19,4.0,33.0,82.0,0.0,62.0,38.0,31.0,42.0,12.0,56.0,11.0,26.0,8.0,19.0,23.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Danzas españolas - 3 Zarabanda,Enrique Granados,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,12.0,12.0,12.0,0.0,27.0,97.0,0.0,28.0,0.0,34.0,14.0,6.0,75.0,16.0,48.0,0.0,29.0,94.0,0.0,84.0,0.0,169.0,49.0,0.0,75,24.0,75,26.0,91.0,282,0.0,143.0,4.0,170.0,188.0,0.0,46,8.0,83.0,12.0,90.0,135.0,0.0,61.0,8.0,51.0,42.0,0.0,28.0,0.0,29.0,1.0,13.0,32.0,0.0,7.0,0.0,7.0,4.0,0.0,2.0,0.0,1.0,1.0,3.0,4.0,0.0,2.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0
5,Thunderstorm,Burgmueller,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,11.0,4.0,35.0,13.0,0.0,7.0,10.0,65.0,6.0,10.0,26.0,2.0,11.0,18.0,57.0,11.0,4.0,8.0,16.0,98.0,6.0,31.0,47.0,11.0,34.0,4.0,72,25.0,2,18.0,11.0,81,0.0,15.0,15.0,3.0,7.0,0.0,10,2.0,0.0,2.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"Sonata No. 14 C# minor (Moonlight) , Opus 27/2...",Beethoven,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,12.0,1.0,39.0,3.0,1.0,16.0,14.0,41.0,2.0,7.0,15.0,3.0,18.0,4.0,79.0,5.0,1.0,16.0,14.0,39.0,3.0,14.0,44.0,3.0,43.0,16.0,80.0,44,2.0,64,32.0,109.0,14,58.0,87.0,14.0,87.0,20.0,85.0,42,5.0,30.0,19.0,43.0,5.0,11.0,10.0,0.0,4.0,1.0,2.0,3.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"Songs without Words Book 2, Opus 30 - Part 1",Mendelssohn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,8.0,0.0,26.0,6.0,16.0,10.0,2.0,29.0,0.0,6.0,4.0,20.0,27.0,0.0,72.0,6.0,36.0,10.0,38.0,109.0,0.0,78.0,16.0,109.0,87.0,8,123.0,36,38.0,24.0,57,95.0,0.0,83.0,38.0,47.0,99.0,2,62.0,52.0,14.0,12.0,19.0,41.0,0.0,14.0,8.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"Piano Sonata in C major, Hoboken XVI:7 - Movem...",Haydn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,10.0,0.0,12.0,0.0,12.0,14.0,0.0,20.0,0.0,18.0,12.0,4.0,42.0,0.0,12,0.0,60,68.0,0.0,70

Para obtener el espacio que ocupa el dataframe sin optimziar utilizamos el método info con el argumento memory_usage='deep' para obtener todo
el espacio que ocupan todos los elementos del dataframe.

In [46]:
df.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 90 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TITULO  30 non-null     object 
 1   AUTOR   30 non-null     object 
 2   A0      30 non-null     float64
 3   A#0     30 non-null     float64
 4   B0      30 non-null     float64
 5   C1      30 non-null     float64
 6   C#1     30 non-null     float64
 7   D1      30 non-null     float64
 8   D#1     30 non-null     float64
 9   E1      30 non-null     float64
 10  F1      30 non-null     float64
 11  F#1     30 non-null     float64
 12  G1      30 non-null     float64
 13  G#1     30 non-null     float64
 14  A1      30 non-null     float64
 15  A#1     30 non-null     float64
 16  B1      30 non-null     float64
 17  C2      30 non-null     float64
 18  C#2     30 non-null     float64
 19  D2      30 non-null     float64
 20  D#2     30 non-null     float64
 21  E2      30 non-null     float64
 22  F2  

El dataframe original ocupa 25.6 KB.

Para su optimización, primeramente, se configuran los tipos de datos idoneos para cada tipo de columna y no los inferidos por pandas. Se puede observar que el tipo de dato inferido para las notas es un float de 64 bits. Si se analizan estos datos se obtiene que los números son números enteros, sin signo y que no tienen un rango tan alto. Por estos motivos tiene sentido modificar el tipo de dato de estas columnas a enteros de 16 bits, que necesitan menos espacio.
Además, el tipo de dato object se ha modificado al tipo de dato string para disminuir el espacio necesario.

Para modificar el tipo de dato de las columnas primero se ha obtenido el nombre de las columnas utilizando el método select_dtypes y posteriormente se ha utilizado el método astype que permite cambiar el tipo de dato.

In [47]:
s = df.select_dtypes(include=['float64', 'int64']).columns
df[s] = df[s].astype("int16")

s = df.select_dtypes(include='object').columns
df[s] = df[s].astype("string")

In [48]:
df.dtypes

TITULO    string
AUTOR     string
A0         int16
A#0        int16
B0         int16
C1         int16
C#1        int16
D1         int16
D#1        int16
E1         int16
F1         int16
F#1        int16
G1         int16
G#1        int16
A1         int16
A#1        int16
B1         int16
C2         int16
C#2        int16
D2         int16
D#2        int16
E2         int16
F2         int16
F#2        int16
G2         int16
G#2        int16
A2         int16
A#2        int16
B2         int16
C3         int16
C#3        int16
D3         int16
D#3        int16
E3         int16
F3         int16
F#3        int16
G3         int16
G#3        int16
A3         int16
A#3        int16
B3         int16
C4         int16
C#4        int16
D4         int16
D#4        int16
E4         int16
F4         int16
F#4        int16
G4         int16
G#4        int16
A4         int16
A#4        int16
B4         int16
C5         int16
C#5        int16
D5         int16
D#5        int16
E5         int16
F5         int

Comprobamos que el tamaño del dataframe ha disminuido tras aplicar los cambios.

In [49]:
df.info(verbose=False, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Columns: 90 entries, TITULO to C8
dtypes: int16(88), string(2)
memory usage: 10.1 KB


EL dataframe ahora ocupa 10.1 KB, lo que supone un ahorro de 15,5 KB respecto al tamaño original. En otras palabras, se ha conseguido reducir en un 39,45% el tamaño del dataframe.

3. Queremos obtener respuesta a las siguientes preguntas:

In [50]:
notes_cols = list(df.columns)
notes_cols.remove('TITULO')
notes_cols.remove('AUTOR')

a) Averiguar cuál es la tecla más popular de cada compositor individual y de todos los
compositores de forma conjunta. Es decir: la tecla del piano que más veces aparece en
todas las partituras de un compositor y en todas las partituras en global.

Tecla más popular de cada compositor de forma individual. Tenemos que agrupar por la columna AUTOR y seleccionar la columna que tiene el valor mayor para cada uno.

In [51]:
df.groupby('AUTOR').sum().idxmax(axis=1)

AUTOR
Beethoven                E4
Burgmueller              D3
Chopin                  G#4
Debussy                 F#4
Enrique Granados         D4
Haydn                    G4
Johan Sebastian Bach     C4
Mendelssohn              G3
Tchaikovsky              D4
dtype: object

Tecla más popular de forma global. Tenemos que realizar el sumatorio de las columnas con el método sum y obtener la columna que tiene el valor más mayor.

In [52]:
df.sum().idxmax()

'D4'

b) Averiguar cuál es la tecla más odiada de cada compositor individual y de todos los
compositores de forma conjunta. Es decir: la tecla del piano que menos veces aparece
en todas las partituras de un compositor y en todas las partituras en global.

Tecla más odiada de cada compositor individual. En primer lugar agrupamos el dataframe por la columna AUTOR y calculamos la suma de sus notas. Tras eso, realizamos un filtrado de los datos para quedarnos solamente con aquellas notas que aparecen al menos 1 vez, mientras que el resto toman el valor NA. El método idxmin permite obtener el id de la columna con el valor más pequeño y, con el argumento skipna, no teniendo en cuenta aquellas columnas que tiene un valor nulo. 

In [53]:
df_grouped = df.groupby('AUTOR').sum()

df_grouped.where(df_grouped > 0).idxmin(axis=1, skipna=True)

AUTOR
Beethoven                F1
Burgmueller              E1
Chopin                   C1
Debussy                 G#1
Enrique Granados         B6
Haydn                    D2
Johan Sebastian Bach     E2
Mendelssohn             G#5
Tchaikovsky             C#2
dtype: object

Tecla más odiada de todos los compositores de forma conjunta. El planteamiento es el mismo que el apartado anterior, salvo que no se realiza un agrupamiento por la columna AUTOR.

In [54]:
df_sum = df.sum()

df_sum.where(df_sum > 0).idxmin(axis=0, skipna=True)

'C1'

c) Averiguar cuál es la octava favorita de cada compositor individual y de todos los
compositores de forma conjunta. Es decir: la octava del piano (de 0 a 8) que más notas
acumula entre todas las partituras de un compositor y entre todas las partituras en
global.

En primer lugar, calculamos las octavas para cada composición utilizando un bucle for que permita obtener el nombre de las columnas correspondientes para cada octava y posteriormente se calcula su total de notas con el método sum.

In [55]:
for i in range(0,9):
    octave = [col for col in df.columns if str(i) in col]
    octave_str = f'octava_{i}'
    df[octave_str] = df[octave].sum(axis=1)

In [56]:
df

,TITULO,AUTOR,A0,A#0,B0,C1,C#1,D1,D#1,E1,F1,F#1,G1,G#1,A1,A#1,B1,C2,C#2,D2,D#2,E2,F2,F#2,G2,G#2,A2,A#2,B2,C3,C#3,D3,D#3,E3,F3,F#3,G3,G#3,A3,A#3,B3,C4,C#4,D4,D#4,E4,F4,F#4,G4,G#4,A4,A#4,B4,C5,C#5,D5,D#5,E5,F5,F#5,G5,G#5,A5,A#5,B5,C6,C#6,D6,D#6,E6,F6,F#6,G6,G#6,A6,A#6,B6,C7,C#7,D7,D#7,E7,F7,F#7,G7,G#7,A7,A#7,B7,C8,octava_0,octava_1,octava_2,octava_3,octava_4,octava_5,octava_6,octava_7,octava_8
0,"The Seasons, Opus 37a - April - Snowdrop",Tchaikovsky,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,10,2,5,4,21,25,2,36,2,49,17,14,77,15,115,20,110,97,5,57,50,128,77,46,149,16,38,14,47,23,4,17,9,23,8,4,8,0,0,1,14,9,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,557,649,93,4,0,0
1,"The Seasons, Opus 37a - May - Starlight Nights",Tchaikovsky,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,15,12,2,21,3,6,4,28,42,8,35,12,56,55,11,58,22,58,23,48,96,5,35,19,85,72,4,56,37,68,11,25,64,9,23,5,16,14,1,12,2,4,0,0,14,0,0,0,0,0,0,6,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,61,389,548,186,20,2,0
2,"The Seasons, Opus 37a - July - Song of the Reaper",Tchaikovsky,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,15,0,1,1,0,4,3,24,1,0,48,0,25,0,15,60,0,24,3,20,6,44,102,5,72,4,59,97,0,33,7,108,34,14,71,11,51,2,34,41,0,27,4,28,6,2,8,2,7,2,5,5,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,97,304,510,205,21,0,0
3,"Piano Sonata in C major, Hoboken XVI:7 - Movem...",Haydn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,19,0,0,0,0,24,0,20,11,12,10,2,51,0,4,0,22,33,0,21,6,12,7,4,45,0,19,4,33,82,0,62,38,31,42,12,56,11,26,8,19,23,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,156,184,387,34,0,0
4,Danzas españolas - 3 Zarabanda,Enrique Granados,0,0,0,0,0,0,0,0,0,0,6,6,12,12,12,0,27,97,0,28,0,34,14,6,75,16,48,0,29,94,0,84,0,169,49,0,75,24,75,26,91,282,0,143,4,170,188,0,46,8,83,12,90,135,0,61,8,51,42,0,28,0,29,1,13,32,0,7,0,7,4,0,2,0,1,1,3,4,0,2,0,3,2,0,0,0,0,0,0,48,345,599,1041,456,67,15,0
5,Thunderstorm,Burgmueller,0,0,0,0,0,19,0,0,0,0,11,4,35,13,0,7,10,65,6,10,26,2,11,18,57,11,4,8,16,98,6,31,47,11,34,4,72,25,2,18,11,81,0,15,15,3,7,0,10,2,0,2,0,6,0,0,6,0,0,0,0,0,4,0,0,4,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,82,227,354,162,18,6,0,0
6,"Sonata No. 14 C# minor (Moonlight) , Opus 27/2...",Beethoven,0,0,0,0,2,0,0,0,1,12,1,39,3,1,16,14,41,2,7,15,3,18,4,79,5,1,16,14,39,3,14,44,3,43,16,80,44,2,64,32,109,14,58,87,14,87,20,85,42,5,30,19,43,5,11,10,0,4,1,2,3,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,75,205,366,583,99,3,0,0
7,"Songs without Words Book 2, Opus 30 - Part 1",Mendelssohn,0,0,0,0,0,0,0,0,2,4,0,8,0,26,6,16,10,2,29,0,6,4,20,27,0,72,6,36,10,38,109,0,78,16,109,87,8,123,36,38,24,57,95,0,83,38,47,99,2,62,52,14,12,19,41,0,14,8,2,1,0,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,192,650,597,113,3,0,0
8,"Piano Sonata in C major, Hoboken XVI:7 - Movem...",Haydn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,10,0,12,0,12,14,0,20,0,18,12,4,42,0,12,0,60,68,0,70,4,26,8,2,128,0,8,0,26,37,0,64,6,50,30,16,46,0,20,0,12,8,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38,182,340,281,12,0,0
9,Wedding March,Mendelssohn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,15,1,27,0,8,0,8,15,0,0,0,14,15,1,29,0,8,0,8,76,0,6,0,44,18,1,64,0,24,0,53,101,1,79,16,50,9,32,21,0,9,0,9,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71,90,286,327,8,0,0


Una vez calculadas las octavas, obtenemos las columnas correspondientes para poder hacer la selección únicamente de las columnas que se necesitan. Añadimos también la columna AUTOR porque también es necesaria.

In [57]:
octaves = [f'octava_{str(i)}' for i in range(0,9)]
octaves.append('AUTOR')

Octava más popular de forma individual. Se seleccionan las octavas, se agrupa por la columna AUTOR, se realiza el sumatorio de las filas y se obtiene el id de la columna con el valor mayor.

In [58]:
df[octaves].groupby('AUTOR').sum().idxmax(axis=1)

AUTOR
Beethoven               octava_4
Burgmueller             octava_5
Chopin                  octava_4
Debussy                 octava_4
Enrique Granados        octava_4
Haydn                   octava_5
Johan Sebastian Bach    octava_4
Mendelssohn             octava_4
Tchaikovsky             octava_4
dtype: object

Octava más popular de forma conjunta. El plantemiento es el mismo que el apartado anterior salvo que en este caso no se realiza la agrupación por AUTOR.

In [59]:
df[octaves].sum().idxmax()

'octava_4'

d) Averiguar cuál es la octava más odiada de cada compositor individual y de todos los
compositores de forma conjunta. Es decir: la octava del piano (de 0 a 8) que menos
notas acumula entre todas las partituras de un compositor y entre todas las partituras en
global.

Octava más odiada de cada compositor individual. Primero seleccionamos las columnas necesarias y agrupamos los datos por la columna AUTOR. Tras esto, realizamos la suma para obtener el total de las octavas. Para filtrar aquellas octavas que al menos tienen una nota filtramos los resultados utilizando el método where. Por último, se utiliza el método idxmin para obtener el id de la columna con un valor más pequeño sin tener en cuenta los valores nulos.

In [60]:
df_sum = df[octaves].groupby('AUTOR').sum()
df_sum.where(df_sum > 0).idxmin(axis=1, skipna=True)

AUTOR
Beethoven               octava_7
Burgmueller             octava_7
Chopin                  octava_7
Debussy                 octava_7
Enrique Granados        octava_7
Haydn                   octava_6
Johan Sebastian Bach    octava_6
Mendelssohn             octava_6
Tchaikovsky             octava_7
dtype: object

Octava más odiada de todos los compositores de forma conjunta. El planteamiento es el mismo que en el apartado anterior, salvo que no se realiza la agrupación por la columna AUTOR.

In [61]:
df_sum = df[octaves].sum()

df_sum.where(df_sum > 0).idxmin(axis=0, skipna=True)

'octava_7'

4. Sabiendo que, en términos musicales, la distancia entre dos teclas consecutivas del piano se denomina semitono (que es la mitad de un tono), queremos averiguar cual es la amplitud de cada partitura (que definiremos como la distancia entre la tecla más a la izquierda utilizada en la partitura y la tecla más a la derecha utilizada en la partitura) medida en tonos.

4.a) Programa un método para obtener la nota más aguda (la tecla situada más a la derecha)
que se utiliza en una partitura. 
Para resolver esta parte se ha creado un método, get_highest_note, que recibe un objeto serie de pandas, que equivale a una partitura y que devuelve el la nota más aguda. Utilizando el método where filtramos aquellas notas que aparecen al menos una vez, mientras que las que no aparecen toman el valor NA. El método last_valid_index permite obtener aquel índice, columna, que tiene un valor distinto de NA. De esta forma obtenemos la nota más a la derecha del piano que aparece en la partitura.

In [62]:
def get_highest_note(serie):
    return serie.where(serie > 0).last_valid_index()

El método apply permite ejecutar la función anterior a todas las filas, partituras, del dataframe.

In [63]:
df['highest_note'] = df[notes_cols].apply(get_highest_note, axis=1)

4.b) Programa un método para obtener la nota más grave (la tecla situada más a la izquierda)
que se utiliza en una partitura.
El razonamiento de este apartado es el mismo que el anterior salvo que se obtiene el primer índice válido, es decir, la nota situada más a la izquierda del piano.

In [64]:
def get_lowest_note(serie):
    return serie.where(serie > 0).first_valid_index()

El cálculo de esta nota también se realiza ejecutando el método apply.

In [65]:
df['lowest_note'] = df[notes_cols].apply(get_lowest_note, axis=1)

4.c) Programa un método que calcule la amplitud de la partitura medida en tonos.

En primer lugar, se crea un diccionario, note_cols_dict, que contiene como llave una nota del piano y como valor su posición en el mismo. Por ejemplo, la nota A0, que es la primera, tiene el valor 0, la siguiente nota, A#0 el valor 1, B0 el 2, C1 el 3, etc... De esta forma podemos obtener la posición de una nota en el piano de una forma computancionalmente sencilla porque el acceso al diccionario es O(1). 

In [66]:
note_cols_dict = {k:v for v,k in enumerate(notes_cols)}

Tras esto, desarrollamos el método get_amplitude que recibe un objeto serie de pandas y el diccionario anterior y permite calcular la amplitud de una partitura. Para obtener la nota de mayor amplitud se utiliza la columna highest_note mientras que para obtener la nota de menor amplitud se utiliza la columna lowest_note. Al acceder al diccionario utilizando como clave una nota se obtiene su posición. El cálculo de la ampltud se realiza restando la posición de la nota menor a la posición de la nota mayor y diviendo el resultado entre 2. Como el resultado puede no ser entero y este caso es correcto no se emplea ninguna transformación de datos.

In [67]:
def get_amplitude(serie, note_cols_dict) -> float:
    return (note_cols_dict[serie['highest_note']] - note_cols_dict[serie['lowest_note']]) / 2

El cálculo de la amplitud para cada partitura se realiza empleando el médo apply. Para optimizar el cálculo solamente se le pasa a este método las columnas necesarias para realizar el cálculo, que son lowest_note y highest_note.

In [68]:
df['amplitude'] = df[['lowest_note', 'highest_note']].apply(get_amplitude, args=(note_cols_dict,), axis=1)

4.d) Ahora deseamos averiguar cual es la partitura con mayor y menor amplitud en general y
para cada compositor. Escribe el código necesario para realizar este cálculo.


Partitura con mayor amplitud en general. Primero seleccionamos las columnas necesarias: AUTOR, TITULO y amplitude. Tras esto, ordenamos los resultados por la columna amplitude de mayor a menor y seleccionamos solamente el primer resultado con el método head.

In [69]:
df[['AUTOR','TITULO', 'amplitude']].sort_values(by='amplitude', ascending=False).head(1)

,AUTOR,TITULO,amplitude
23,Chopin,"Polonaise Ab major, Opus 53",37.5


Partitura con menor amplitud en general. El planteamiento es el mismo que el apartado anterior, salvo que ordenamos los valores de menor a mayor.

In [70]:
df[['AUTOR','TITULO', 'amplitude']].sort_values(by='amplitude').head(1)

,AUTOR,TITULO,amplitude
27,Beethoven,"Symphony No. 9 in Dm, 4th movement ""Ode to Joy...",12.0


Partitura con mayor amplitud para cada compositor. Tras ordenar los resultados de forma descendente agrupamos los resultados por la columna AUTOR y seleccionamos solamente la primera fila para cada compositor con el método head. Por último se ordenan los resultados alfabéticamente en base al AUTOR.

In [71]:
df[['AUTOR','TITULO', 'amplitude']].sort_values(by='amplitude', ascending=False).groupby('AUTOR').head(1).sort_values(by='AUTOR')

,AUTOR,TITULO,amplitude
25,Beethoven,For Elise,33.5
26,Burgmueller,Spinning Song,36.0
23,Chopin,"Polonaise Ab major, Opus 53",37.5
22,Debussy,Clair de Lune,35.0
4,Enrique Granados,Danzas españolas - 3 Zarabanda,36.0
10,Haydn,"Piano Sonata in C major, Hoboken XVI:7 - Movem...",25.0
13,Johan Sebastian Bach,Prelude and Fugue in C major BWV 846,24.0
7,Mendelssohn,"Songs without Words Book 2, Opus 30 - Part 1",30.0
19,Tchaikovsky,"The Seasons, Opus 37a - June - Barcarolle",36.0


Partitura con menor amplitud para cada compositor. El razonamiento es el mismo que el apartado anterior, salvo que las partituras se ordenan de forma ascendente y se selecciona la primera de cada autor. 

In [72]:
df[['AUTOR','TITULO', 'amplitude']].sort_values(by='amplitude').groupby('AUTOR').head(1).sort_values(by='AUTOR')

,AUTOR,TITULO,amplitude
27,Beethoven,"Symphony No. 9 in Dm, 4th movement ""Ode to Joy...",12.0
5,Burgmueller,Thunderstorm,31.5
21,Chopin,"Grande Valse Brillante, Opus 18",36.0
22,Debussy,Clair de Lune,35.0
16,Enrique Granados,Danzas españolas - 2 Oriental,27.5
3,Haydn,"Piano Sonata in C major, Hoboken XVI:7 - Movem...",25.0
13,Johan Sebastian Bach,Prelude and Fugue in C major BWV 846,24.0
9,Mendelssohn,Wedding March,22.0
28,Tchaikovsky,"The Seasons, Opus 37a - October - Autumn Song",22.5


5. Crea el código necesario para generar, de forma aleatoria, archivos con partituras en forma de fichero de texto como el utilizado en los ficheros de entrada. Ten en cuenta que en estos archivos pueden aparecer las notas extrañas (E# y B#), pero todas deberán estar en el rango de notas válido para un piano

En primer lugar vamos a realizar un pequeño análisis estadístico para que las partituras creadas sean más realistas. Se va a calcular el número de notas de las partituras de muestra para obtener un rango de notas adecuado.

In [73]:
df['total_notes'] = df[notes_cols].apply(sum, axis=1)
df['total_notes'].sort_values(ascending=False)

23    6341
21    4456
24    2691
4     2571
15    2559
18    2317
17    2057
20    2001
29    1889
22    1750
14    1690
19    1678
7     1601
16    1525
13    1404
0     1375
6     1331
1     1206
26    1161
2     1147
25    1020
11     961
28     934
8      853
5      849
3      786
9      782
12     625
10     507
27     269
Name: total_notes, dtype: int64

La consulta anterior nos permite establecer el rango de una partitura: entre 6341 y 269 notas.

In [74]:
notes_min = df['total_notes'].sort_values(ascending=True).head(1).item()
notes_max = df['total_notes'].sort_values(ascending=False).head(1).item()

Tras esto, calculamos la lista de notas que aparecen al menos una vez en las partituras. Esto se realiza cargando las notas de todas las partituras y sumando el número de veces que aparece cada una de ellas. Como los campos AUTOR y TITULO no nos intesan los borramos del diccionario. El resultado se guarda en la variable notas_dict. Una vez realizado este cálculo transformamos este diccionario en una lista y nos quedamos solamente con las notas. 

In [75]:
notes_dict = dict()

input_files = os.listdir('PARTITURAS/')
for input_file in input_files:
    path = os.path.join('PARTITURAS/', input_file)
    notes = load_data(path)
    del notes['AUTOR']
    del notes['TITULO']
    notes_dict = {**notes_dict, **notes}

notes_list = [k for k,_ in notes_dict.items()]

Con el fin de flexibilizar la creación de nuevas partituras creamos un método que reciba como parámetro el número de partituras que se quieren crear, así como las variables calculadas en este apartado para crear las partituras. Las partituras van a tener un número de notas entre el rango antes calculado. 

La generación de números aleatorios se realiza con la librería random. Como vamos a acceder a una posición de la lista de notas necesitamos un valor entero, por ese motivo se utiliza el método randint. El rango del número aleatorio se establece entre el 0, la primera posición de la lista de notas, y la longitud de la lista menos 1, que es su última posición. Los valores extremos también están incluidos.

In [76]:
def create_score(num_score:int, notes_list: List['str'], notes_min:int, notes_max: int):
    for i in range(0, num_score):
        with open(f'partitura_jgll{i}.txt', '+w') as file:
            file.write('jgalan279\n')
            file.write(f'partitura{i}\n')
            notes_random = random.randint(notes_min, notes_max)
            for _ in range(0, notes_random):
                rad = random.randint(0, len(notes_list) -1 )
                file.write(f'{notes_list[rad]}\n')

Por último, llamamos al método anterior con los argumentos que deseemos.

In [77]:
create_score(20, notes_list, notes_min, notes_max)

## Conclusiones

Esta práctica me ha resultado muy interesante y me ha permitido mejorar y poner en práctica los conocimientos adquiridos en esta asignatura. Me ha resultado muy útil especialmente para comprender mejor el uso de la librería pandas y de las múltiples funciones que ofrece. Sin duda considero que es una herramienta que cualquier profesional que esté en áreas relacionadas con el tratamiento de datos debería de conocer. 

El tema de la práctica me ha resultado muy original y ha hecho que su desarrollo me resultara más ameno. Sin embargo, creo que al limitar el tema de la práctica las preguntas que se podían hacer también quedaban limitadas y no se ha podido ver otras funciones de pandas o de la librería numpy que también considero interesantes. Como recomendación se podría modificar la práctica para dividirla en un número de ejercicios independientes en el que cada uno sirviera para prácticar con distintas funcionalidades de las librerías impartidas en la asignatura. 

También fecilitar al equipo docente por animarnos a utilizar el foro de la asignatura y por su atención ante las dudas que hemos planteado. 

En definitiva, considero que esta práctica me ha resultado de una gran utilidad y estoy satisfecho del resultado final.
